<a href="https://colab.research.google.com/github/tushjai97/PDF-query-using-LLM/blob/main/PDF_query_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Querying PDF with Astra and Langchain


In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 691.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

###Setup

##Provide Your Secrets

Replace the following with your Astra DB connection details and your OpenAPI key


In [ ]:
ASTRA_DB_APPLICATION_TOKEN = " "
ASTRA_DB_ID = " "
OPENAI_API_KEY = " "

In [ ]:
pdfreader = PdfReader("EmergencyPandemic Preparedness Toolkit (4)_0_0.pdf")

In [ ]:
from typing_extensions import Concatenate

raw_text = ''
for i,page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text +=content


In [ ]:
raw_text

'Emergency & Pandemic\nPreparedness Toolkit\nBe prepared for your family\'s unique needs in a crisis.\nStay informed. Make a plan. Be prepared.Overview\nTable of Contents:A n  e m e r g e n c y  o r  p a n d e m i c  c a n  b e  e s pe c i a l l y  c ha l l e n g i n g  f o r  f a m i l i e s  o f  c hi l d r e n\nw i t h  s p e c i a l  h e a l t h c a r e  n e e d s .  W he n  i t  c o m e s  t o  emergency  p r e p a r e d n e s s  i t \' s\ni m p o r t a n t  f o r  f a m i l i e s  o f  c h i l d r e n  w i t h d i s a b i l i t i e s  t o  pl a n  f o r  n a t u r a l  d i s a s t e r s ,\ne s p e c i a l l y  l i v i n g  i n  C a l i f o r n i a  w h e r e  f i r e s  a n d  e a r t h q u a k e s  c o m m o n l y  o c c u r .\nA d d i t i o n a l l y , c h i l d r e n  w i t h  s p e c i a l  he a l t h c a r e  n e e d s  a r e  m o r e  l i k e l y  t o  e x pe r i e n c e\no n g o i n g  d i s r u p t i o n s  i n  h e a l t h c a r e , e d u c a t i o n , a n d  o t he r  e

## Initialize the connection to database

In [ ]:
cassio.init(token = ASTRA_DB_APPLICATION_TOKEN, database_id = ASTRA_DB_ID)

Create Langchain embedding

In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Create Langchain vector store ... backed by Astra DB!

In [ ]:
#Storing Embeddings in Cassandra
astra_vector_store= Cassandra(
    embedding = embedding,
    table_name = 'qa_mini_demo',
    session = None,
    keyspace= None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)

texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

["Emergency & Pandemic\nPreparedness Toolkit\nBe prepared for your family's unique needs in a crisis.\nStay informed. Make a plan. Be prepared.Overview\nTable of Contents:A n  e m e r g e n c y  o r  p a n d e m i c  c a n  b e  e s pe c i a l l y  c ha l l e n g i n g  f o r  f a m i l i e s  o f  c hi l d r e n\nw i t h  s p e c i a l  h e a l t h c a r e  n e e d s .  W he n  i t  c o m e s  t o  emergency  p r e p a r e d n e s s  i t ' s\ni m p o r t a n t  f o r  f a m i l i e s  o f  c h i l d r e n  w i t h d i s a b i l i t i e s  t o  pl a n  f o r  n a t u r a l  d i s a s t e r s ,\ne s p e c i a l l y  l i v i n g  i n  C a l i f o r n i a  w h e r e  f i r e s  a n d  e a r t h q u a k e s  c o m m o n l y  o c c u r .",
 'e s p e c i a l l y  l i v i n g  i n  C a l i f o r n i a  w h e r e  f i r e s  a n d  e a r t h q u a k e s  c o m m o n l y  o c c u r .\nA d d i t i o n a l l y , c h i l d r e n  w i t h  s p e c i a l  he a l t h c a r e  n e e d s  a r e  m o r 

Loading data into the vetor store

In [ ]:
astra_vector_store.add_texts(texts[:50])
print("inserted %i headlines." %len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

inserted 49 headlines.


In [ ]:
first_question = True
while True:
  if first_question:
    query_text = input("\n Enter your question (or type 'quite' to exit): ").strip()
  else:
    query_text = input("\n What is your next question (or type 'quit' to exit): ").strip()

  if query_text == "":
    continue
  first_question = False
  print("\nQUESTION: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \"%s\"\n" % answer)

  print("FIRST DOCUMENTS BY RELEVANCE: ")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text,k=4):
    print(" [%0.4f] \"%s...\""%(score, doc.page_content[:84]))




QUESTION: "what happens in a pandemic"


ANSWER: "A pandemic is an outbreak of a disease that spreads across the globe. Pandemics are hard to predict, can start anywhere, and spread very quickly. Diseases can be spread directly from person to person, from germs on objects, and from people who don't look sick. In order to prepare for a pandemic, it is important to become informed and have a plan in place. This can include updating your emergency kit and having emergency contacts and medical information readily available. During a pandemic, schools, workplaces, and community centers may also close as a precautionary measure."

FIRST DOCUMENTS BY RELEVANCE: 


 [0.9262] "d i s e a s e  i s  s p r e a d .  A  pandemic  i s  a n  o u t b r e a k  o f  a  d..."
 [0.9261] "d i s e a s e  i s  s p r e a d .  A  pandemic  i s  a n  o u t b r e a k  o f  a  d..."
 [0.9172] "E m e r g e n c y  C o n t a c t s . . . . . . . . . . . . . . . . . . . . . . . . ...."
 [0.9172] "E m e r g e n c y  C o n t a c t s . . . . . . . . . . . . . . . . . . . . . . . . ...."
